In [3]:
# Use OpenCL (This Way Hides Details)

import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy as np  # Import Numpy number tools

context = cl.create_some_context()  # Initialize the Context
queue = cl.CommandQueue(context)  # Instantiate a Queue
n=5
m=5
thr = 3

#part_c = np.random.rand(n,3).astype(np.float32)
#probe_c = np.random.rand(m,3).astype(np.float32)
#e_c = np.zeros((m, 3)).astype(np.float32)

part_c = np.array([[1,2,3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]).astype(np.float32)#
probe_c = np.array([[2,4,6], [8, 10, 12], [14, 16, 18], [20, 22, 24], [16, 17, 18]]).astype(np.float32)#
e_c = np.zeros((m, 3)).astype(np.float32)
charge_n = np.array([5,5,5,5,5]).astype(np.float32)

part = pycl_array.to_device(queue, part_c)
probe = pycl_array.to_device(queue, probe_c)
e = pycl_array.to_device(queue, e_c)

charge = pycl_array.to_device(queue, charge_n)

program = cl.Program(context, """
__kernel void calc_field(__global const float *part, __global const float *probe, __global const float *charge, __global float *e, const int thr, const int n)
{
  int j = get_global_id(1);
  int i = get_global_id(0);
  for(int k = 0; k<n; k++){
    e[i*thr + j] += charge[k]/(probe[i*thr + j] - part[k*thr + j]);
  }      
  //float k_0 = pow(10.0,9.0);
  //float k = 9*k_0;  
}""").build()  # Create the OpenCL program

program.calc_field(queue, probe.shape, None, part.data, probe.data, charge.data, e.data, np.int32(thr), np.int32(n))# Enqueue the program for execution and store the result in c

for j in range(probe_c.shape[0]):#m2
  for i in range(probe_c.shape[1]):#m3
    for k in range(part_c.shape[0]):#n5
      e_c[j][i] += charge_n[k]/(probe_c[j][i] - part_c[k][i])

#k = 9*10**9;
print("particle_position: {}".format(part))
print("probe_position: {}".format(probe))
print("charge: {}".format(charge))
print("out1: {}".format(e))
print("out2: {}".format(e_c))

# Print all three arrays, to show calc_field() worked

particle_position: [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]
 [13. 14. 15.]]
probe_position: [[ 2.  4.  6.]
 [ 8. 10. 12.]
 [14. 16. 18.]
 [20. 22. 24.]
 [16. 17. 18.]]
charge: [5. 5. 5. 5. 5.]
out1: [[ 0.42045453 -4.964286           inf]
 [ 3.4642859  -2.125              inf]
 [ 7.8489013   4.9366884   3.8055556 ]
 [ 2.174559    1.980806    1.8214287 ]
 [ 3.8055556   3.8055556   3.8055556 ]]
out2: [[ 0.42045453 -4.964286           inf]
 [ 3.4642859  -2.125              inf]
 [ 7.8489013   4.9366884   3.8055553 ]
 [ 2.174559    1.980806    1.8214285 ]
 [ 3.8055553   3.8055553   3.8055553 ]]


/home/fizmat/Desktop/ef_python/venv/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in float_scalars


In [2]:
# Use OpenCL (This Way Hides Details)

import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array (a Numpy array plus an OpenCL buffer object)
import numpy as np  # Import Numpy number tools
import math
context = cl.create_some_context()  # Initialize the Context
def region_populate_particles(reg):
    q = 1
    m = 2
    #pos = np.array([[1,2,3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]).astype(np.float32)
    pos = np.random.rand(reg[3], 3).astype(np.float32)
    vel = np.random.rand(reg[3], 3).astype(np.float32)
    vel = vel - reg[5] / 2.0 # both positive and negative directions
    vel = vel / 10 # make it smaller
    #vel = np.zeros((reg[3], 3)).astype(np.float32)#for example
    pos_c = pos;
    vel_c = vel;
    particles = []
    for n in range(reg[3]):
        p = [n, q, m]
        particles.append(p)
    part_c = particles;
    return particles, pos, vel, pos_c, vel_c, part_c


def region_simulate(reg, particles, pos, vel, pos_c, vel_c, part_c):
    queue = cl.CommandQueue(context)  # Instantiate a Queue
    pos_d = pycl_array.to_device(queue, pos)
    vel_d = pycl_array.to_device(queue, vel)
    part_d = pycl_array.to_device(queue, np.asarray(particles))
    
    for t in range(reg[1]):
        print("from t = {} to {} of {}".format(t, t+1, reg[1]))
        print("n_of_particles = ", particles[3])
        pos_c, vel_c = region_advance_one_time_step_host(reg, pos_c, vel_c, part_c[t]);
        pos_d, vel_d = region_advance_one_time_step_kernel(queue, reg, vel_d, pos_d, pos_c, vel_c, part_c[t]);
        if t % reg[2] == 0:
            pos = pos_d.get();
            vel = vel_d.get();
            #region_save_simulation(reg, t, particles, pos);
    # save final step
    pos_out = pos_d.get();
    vel_out = vel_d.get();
    print("pos_out: {}".format(pos_out))
    print("pos_c: {}".format(pos_c))
    print("vel_out: {}".format(vel_out))
    print("vel_c: {}".format(vel_c))
    #region_save_simulation(reg, reg[1], particles, pos_1, pos_2)

def region_advance_one_time_step_host(reg, pos_c, vel_c, part_c):
    vel_c = calc_update_v(pos_c, vel_c, reg[0], part_c[1], part_c[2], reg[3]);
    pos_c = calc_update_p(pos_c, vel_c, reg[0], reg[3]);
    pos_c, vel_c = rebound(pos_c, vel_c, reg[5], reg[3]);
    # Enqueue the program for execution and store the result in c
    return pos_c, vel_c
  
def region_advance_one_time_step_kernel(queue, reg, vel_d, pos_d, pos_c, vel_c, part_c):
    program.update_v(queue, pos_c.shape, None, pos_d.data, vel_d.data, np.float32(reg[0]), np.float32(part_c[1]), np.float32(part_c[2]), np.int32(reg[3]));
    program.update_p(queue, pos_c.shape, None, pos_d.data, vel_d.data, np.float32(reg[0]), np.int32(reg[3]));
    program.rebound(queue, pos_c.shape, None, pos_d.data, vel_d.data, np.float32(reg[5]), np.int32(reg[3]));
    # Enqueue the program for execution and store the result in c
    return pos_d, vel_d
 
  

def calc_update_v(pos, vel, dt, q, m, n):
  for i in range(pos.shape[0]):   
    if(i < n):
      epsilon = 0.0001;
      f0 = 0;
      f1 = 0;
      f2 = 0;
      for j in range(n):
        if(i != j):
          diff0 = pos[j][0] - pos[i][0];
          diff1 = pos[j][1] - pos[i][1];
          diff2 = pos[j][2] - pos[i][2];     
          dist = math.sqrt(diff0*diff0 + diff1*diff1 + diff2*diff2);
          diff0 = diff0 / dist;
          diff1 = diff1 / dist;
          diff2 = diff2 / dist;
          dist = max(epsilon, dist);
          f0 += diff0/(dist*dist);
          f1 += diff1/(dist*dist);
          f2 += diff2/(dist*dist);
      vel[i][0] -= q * q * m * dt * f0;
      vel[i][1] -= q * q * m * dt * f1;
      vel[i][2] -= q * q * m * dt * f2;
      
  return vel;

def calc_update_p(pos, vel, dt, n):
  for i in range(pos.shape[0]):   
    pos[i][0] += vel[i][0] * dt;
    pos[i][1] += vel[i][1] * dt;
    pos[i][2] += vel[i][2] * dt;
      
  return pos;

def rebound(pos, vel, size, n):
  for i in range(pos.shape[0]):   
    if(pos[i][0] < 0):
      pos[i][0] = - pos[i][0];
      vel[i][0] = - vel[i][0];
    elif (pos[i][0] > size):
      pos[i][0] = 2*size - pos[i][0];
      vel[i][0] = - vel[i][0];
    if(pos[i][1] < 0):
      pos[i][1] = - pos[i][1];
      vel[i][1] = - vel[i][1];
    elif (pos[i][1] > size):
      pos[i][1] = 2*size - pos[i][1];
      vel[i][1] = - vel[i][1];
    if(pos[i][2] < 0):
      pos[i][2] = - pos[i][2];
      vel[i][2] = - vel[i][2];
    elif (pos[i][2] > size):
      pos[i][2] = 2*size - pos[i][2];
      vel[i][2] = - vel[i][2];
      
  return pos, vel;


program = cl.Program(context, """
__kernel void update_v(__global const float *pos, __global float *vel, float const dt, float const q, float const m, int const n){
 int i = get_global_id(0);
 float epsilon = 0.0001;
 float diff0;
 float diff1;
 float diff2;
 float dist;
 float f0 = 0;
 float f1 = 0;
 float f2 = 0;
 for(int j = 0; j<n; j++){
   if(i != j){
     diff0 = pos[3*j] - pos[3*i];
     diff1 = pos[3*j + 1] - pos[3*i + 1];
     diff2 = pos[3*j + 2] - pos[3*i + 2];     
     dist = sqrt(diff0*diff0 + diff1*diff1 + diff2*diff2);
     diff0 = diff0 / dist;
     diff1 = diff1 / dist;
     diff2 = diff2 / dist;
     dist = max(epsilon, dist);
     f0 += diff0/(dist*dist);
     f1 += diff1/(dist*dist);
     f2 += diff2/(dist*dist);
   }
 }
 vel[3*i] -= q * q * m * dt * f0;
 vel[3*i + 1] -= q * q * m * dt * f1;
 vel[3*i + 2] -= q * q * m * dt * f2;
}

__kernel void update_p(__global float *pos, __global float *vel, float const dt, int const n){
 int i = get_global_id(0);
 pos[3*i] += vel[3*i] * dt;
 pos[3*i + 1] += vel[3*i + 1] * dt;
 pos[3*i + 2] += vel[3*i + 2] * dt;
}

__kernel void rebound(__global float *pos, __global float *vel, float const size, int const n){
 int i = get_global_id(0);

 if(pos[3*i] < 0){
   pos[3*i] = - pos[3*i];
   vel[3*i] = - vel[3*i];
 } else if (pos[3*i] > size) {
   pos[3*i] = 2*size - pos[3*i];
   vel[3*i] = - vel[3*i];
 }
 if(pos[3*i + 1] < 0){
   pos[3*i + 1] = - pos[3*i + 1];
   vel[3*i + 1] = - vel[3*i + 1];
 } else if (pos[3*i + 1] > size) {
   pos[3*i + 1] = 2*size - pos[3*i + 1];
   vel[3*i + 1] = - vel[3*i + 1];
 }
 if(pos[3*i + 2] < 0){
   pos[3*i + 2] = - pos[3*i + 2];
   vel[3*i + 2] = - vel[3*i + 2];
 } else if (pos[3*i + 2] > size) {
   pos[3*i + 2] = 2*size - pos[3*i + 2];
   vel[3*i + 2] = - vel[3*i + 2];
 } 
}
""").build()  # Create the OpenCL program


def region_save_simulation(reg, current_time, particles, pos):
    filename_prefix = "sim_"
    plt.figure()
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("t = {}".format(current_time))
    plt.xlim(0, reg[5])
    plt.ylim(0, reg[5])
    for p in range(len(particles)):
        plt.plot(pos[0][p], pos[1][p], 'o')
    filename_full = filename_prefix + "{}.png".format(current_time)
    print("saving t = {}".format(current_time), "to ", filename_full)
    plt.savefig(filename_full)
    plt.close()


def main():
    dt = 0.005
    n_time_steps = 5
    save_step = 1
    n_of_particles = 5
    dim = 2
    region_sizes = 1.0
    reg = [dt, n_time_steps, save_step, n_of_particles, dim, region_sizes]#Константы в одном массиве
    particles, pos, vel, pos_c, vel_c, part_c = region_populate_particles(reg)#Генерирует массивы
    np.asarray(reg)
    region_simulate(reg, particles, pos, vel, pos_c, vel_c, part_c)


if __name__ == "__main__":
    main()

from t = 0 to 1 of 5
n_of_particles =  [3, 1, 2]
from t = 1 to 2 of 5
n_of_particles =  [3, 1, 2]
from t = 2 to 3 of 5
n_of_particles =  [3, 1, 2]
from t = 3 to 4 of 5
n_of_particles =  [3, 1, 2]
from t = 4 to 5 of 5
n_of_particles =  [3, 1, 2]
pos_out: [[0.5667299  0.01728238 0.04590007]
 [0.06530234 0.5321706  0.90100807]
 [0.08600323 0.18411465 0.48421457]
 [0.36199656 0.99230844 0.661934  ]
 [0.9899629  0.33101773 0.10179843]]
pos_c: [[0.5667299  0.01728238 0.04590007]
 [0.06530234 0.5321706  0.90100807]
 [0.08600323 0.18411463 0.48421457]
 [0.36199656 0.99230844 0.661934  ]
 [0.9899629  0.33101773 0.10179843]]
vel_out: [[-0.04246996 -0.16144887 -0.17823619]
 [-0.15986812 -0.02040527  0.19766799]
 [-0.10848005 -0.12896806 -0.05478285]
 [ 0.07802386  0.26353064 -0.02180871]
 [ 0.26272023  0.12497663 -0.05535208]]
vel_c: [[-0.04246997 -0.16144888 -0.17823619]
 [-0.15986815 -0.02040528  0.19766799]
 [-0.10848005 -0.12896806 -0.05478285]
 [ 0.07802387  0.26353067 -0.02180871]
 [ 0.2627